In [1]:
from PIL import Image, ImageDraw
import numpy as np
from enum import IntEnum
import world_object

In [53]:
COLORS = {
    "red": '#FF000000',
    "green": '#00FF00',
    "blue": '#0000FF',
    "purple": '#A020F0',
    "yellow": '#FFFF00',
    "grey": '#808080',
}

SHAPES = {
    "TRIANGLE":"triangle",
    "SQUARE":"square",
    "CIRCLE":"circle",
    "RECTANGLE":"rectangle"
}
# Map of object type to integers
OBJECT_TO_IDX = {
    "empty": 0,
    "agent": 1,
    "goal": 8,
#     "wall": 9,
#     "door": 4,
#     "key": 5,
#     "ball": 6,
#     "box": 7,
#     "lava": 9,
}
# Map of agent direction indices to vectors
DIR_TO_VEC = [
    # Pointing right (positive X)
    np.array((1, 0)),
    # Down (positive Y)
    np.array((0, 1)),
    # Pointing left (negative X)
    np.array((-1, 0)),
    # Up (negative Y)
    np.array((0, -1)),
]

width = 200
height = 200

In [3]:
X = [[1,2,3],
     [1,2,3],
     [1,2,3]
    ]

In [4]:
class Actions(IntEnum):
    # Turn left, turn right, move forward
    left = 0
    right = 1
    forward = 2

In [92]:
class Triangle:
    '''
        Attributes
        ----------
        distance_from_cooridinates : int
            How far the center should be from the points
        color : str
            A hex representation of the color that triangle should be
        fill : str
            Whether the object should be filled or not
            
        Methods
        -------
        init(self, size=int, color=str, fill=Bool, sound=None)
            Initializes the class
        draw(self, world=ImageDraw, x=int, y=int)
            Draws the triangle to the given Image at the x,y coordinates
        get_coordinates(self, x=int, y=int)
            Given a centerpoint, calculates the points of the triangle
    '''
    # Size is half of a cell i.e. cell = 100, size = 50
    def __init__(self, size, color, fill = False):
        self.distance_from_coordinate = (size/2)
        self.color = color
        self.fill = fill
    
    #Draw a triangle around this point
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        bottom_left,bottom_right,top = self.get_coordinates(x,y)
        world.polygon([bottom_left, bottom_right, top], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1 = x - self.distance_from_coordinate
        x2 = x + self.distance_from_coordinate
        x3 = x
        y1 = y + self.distance_from_coordinate
        y2 = y + self.distance_from_coordinate
        y3 = y - self.distance_from_coordinate
        
        return (x1, y1), (x2, y2), (x3, y3)

class Rectangle():
    def __init__(self, size, color, fill = False):
        self.distance_from_coordinate = (size/2)
        self.color = color
        self.fill = fill
    
    #Draw a triangle around this point
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        top_left, bottom_right  = self.get_coordinates(x,y)
        print(top_left, bottom_right)
        world.rectangle([top_left, bottom_right], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        height = self.distance_from_coordinate/2
        x1, y1 = x - self.distance_from_coordinate, y - height
        x2, y2 = x + self.distance_from_coordinate, y + height
        return (x1, y1), (x2, y2)

class Square():
    def __init__(self, size, color, fill = False):
        self.distance_from_coordinate = (size/2)
        self.color = color
        self.fill = fill
    
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        top_left, bottom_right = self.get_coordinates(x,y)
        world.rectangle([top_left, bottom_right], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1, y1 = x - self.distance_from_coordinate, y - self.distance_from_coordinate
        x2, y2 = x + self.distance_from_coordinate, y + self.distance_from_coordinate
        return (x1, y1), (x2, y2)

class Circle():
    def __init__(self, size, color, fill = False):
        self.distance_from_coordinate = (size/2)
        self.color = color
        self.fill = fill
    
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        bottom_left, top_right = self.get_coordinates(x,y)
        world.ellipse([bottom_left, top_right], fill = self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1, y1 = x - self.distance_from_coordinate, y - self.distance_from_coordinate
        x2, y2 = x + self.distance_from_coordinate, y + self.distance_from_coordinate
        return (x1, y1), (x2, y2) 


    
class Agent:
    def __init__(self, starting_loc, shape = Triangle, color='#FFFF00'):
        self.row, self.col = starting_loc
        if shape == 'Triangle':
            self.shape = shape(height, color)
            
    def draw(self, world, x, y):
        self.shape.draw(world, x, y)
    
    
class GridWorld:
    def __init__(self, size, background_color='#FFFFFF'):
        self.world = np.zeros(shape = (size, size))
        self.agent = Agent()

In [58]:
def draw_2d_world(grid_size):
    image = Image.new(mode='RGB', size=(width, height), color=(255,255,255, 0))
    # Draw some lines
    draw = ImageDraw.Draw(image)
    y_start = 0
    step_size = 20 
    y_end = step_size*grid_size


    for x in range(0, image.width, step_size):
        line = ((x, y_start), (x, y_end))
        draw.line(line, fill=128)

    x_start = 0
    x_end = step_size*grid_size

    for y in range(0, image.height, step_size):
        line = ((x_start, y), (x_end, y))
        draw.line(line, fill=128)

    del draw
    return image, step_size
    


In [93]:
image, step_size = draw_2d_world(11)
size = step_size/2
draw = ImageDraw.Draw(image)

tri = Triangle(size, COLORS['yellow'])
tri.draw(draw, 90, 110)

circ = Circle(size, COLORS['blue'])
circ.draw(draw, 10, 10)

rec = Rectangle(size, COLORS['green'])
rec.draw(draw, 30, 30)

square = Square(size, COLORS['grey'])
square.draw(draw, 70, 70)
image.show()

(25.0, 27.5) (35.0, 32.5)
